<a href="https://colab.research.google.com/github/Vinaypatil-Ev/vinEvPy-GoCoLab/blob/main/Tensorflow/tensorflowPrac15_Making_new_layers_and_model_via_subclassing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

#Making new layers and model via subclassing

In [ ]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, units=32, input_shape=32, name=None):
        super(CustomLayer, self).__init__()
        winit = tf.random_normal_initializer()
        self.w = tf.Variable(winit(shape=(input_shape, units), dtype="float32"),trainable=True)
        binit = tf.zeros_initializer()                                                                                                                          
        self.b = tf.Variable(binit(shape=(units, ), dtype="float32"),trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [ ]:
CustomLayer(4, 2)(tf.ones((2, 2)))

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.0108623 ,  0.04544608,  0.15549019, -0.00733429],
       [ 0.0108623 ,  0.04544608,  0.15549019, -0.00733429]],
      dtype=float32)>

## instead of tf.variable use built in method add_weights

In [ ]:
class CustomLayer2(tf.keras.layers.Layer):
    def __init__(self, units=32, input_shape=32, name=None):
        super(CustomLayer, self).__init__()
        self.w = self.add_weight(shape=(input_shape, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [ ]:
CustomLayer(4, 2)(tf.ones((2, 2)))

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.05506737, -0.01353997,  0.03594418,  0.10485571],
       [ 0.05506737, -0.01353997,  0.03594418,  0.10485571]],
      dtype=float32)>

## for unknown input shape use buid method

In [ ]:
class CustomLayer3(tf.keras.layers.Layer):
    def __init__(self, units):
        super(CustomLayer3, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer = "random_normal",
            trainable = True)
        
        self.b = self.add_weight(
            shape=(self.units, ),
            initializer = "zeros", 
            trainable = True
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [ ]:
CustomLayer3(4)(tf.ones((2, 2)))

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.08039835, -0.09639128, -0.10998103, -0.0307427 ],
       [-0.08039835, -0.09639128, -0.10998103, -0.0307427 ]],
      dtype=float32)>

## Layer of composite layers

In [ ]:
class CustomCompositeLayer(tf.keras.layers.Layer):
    def __init__(self, units=1):
        super(CustomCompositeLayer, self).__init__()
        self.l1 = CustomLayer3(32)
        self.l2 = CustomLayer3(32)
        self.l3 = CustomLayer3(units)
    
    def call(self, inputs):
        x = self.l1(inputs)
        x = tf.nn.relu(x)
        x = self.l2(x)
        x = tf.nn.relu(x)
        return self.l3(x)

In [ ]:
CustomCompositeLayer(4)(tf.ones((2, 2)))

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.00239614,  0.00448852, -0.00289436, -0.00010922],
       [-0.00239614,  0.00448852, -0.00289436, -0.00010922]],
      dtype=float32)>

## add_loss method in call

In [ ]:
class ActivityRegularizer(tf.keras.layers.Layer):
    def __init__(self, rate):
        super(ActivityRegularizer, self).__init__()
        self.rate = rate 

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs


In [ ]:
class LayerWithKernelRegularizer(tf.keras.layers.Layer):
    def __init__(self, units):
        super(LayerWithKernelRegularizer, self).__init__()
        self.dense = tf.keras.layers.Dense(units, kernel_regularizer=tf.keras.regularizers.l2(1e-2))

    def call(self, inputs):
        return self.dense(inputs)

In [ ]:
l = LayerWithKernelRegularizer(4)
l(tf.ones((2, 2)))

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.1426003 , -0.0952816 ,  0.3554337 , -0.10063767],
       [ 0.1426003 , -0.0952816 ,  0.3554337 , -0.10063767]],
      dtype=float32)>

In [ ]:
l.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0044373358>]

## Auto encoder model

In [ ]:
import tensorflow as tf

In [ ]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        x, y = inputs
        batch = tf.shape(x)[0]
        dim = tf.shape(x)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return x + tf.exp(0.5 * y) * epsilon

class Encoder(tf.keras.layers.Layer):
    def __init__(self, outerdim, innerdim, name="Encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.In = tf.keras.layers.Dense(outerdim, activation="relu")
        self.elr1 = tf.keras.layers.Dense(innerdim)
        self.elr2 = tf.keras.layers.Dense(innerdim)
        self.sampling = Sampling()
    
    def call(self, inputs):
        x = self.In(inputs)
        zmean = self.elr1(x)
        zvar = self.elr2(x)
        z = Sampling((zmean, zvar))
        return zmean, zvar, z

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, originaldim, outerdim, name="Decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dlr1 = tf.keras.layers.Dense(outerdim, activation="relu")
        self.Out = tf.keras.layers.Dense(originaldim, activation="sigmoid") 
    
    def call(self, inputs):
        x = self.dlr1(inputs)
        return self.Out(x)

In [ ]:
class VariationAutoEncoder(tf.keras.Model):
    def __init__(self,originaldim, outerdim, innerdim, name="VAE", **kwargs):
        super(VariationAutoEncoder, self).__init__(name=name, **kwargs)
        self.originaldim = originaldim
        self.encoders = Encoder(outerdim, innerdim)
        self.decoders = Decoder(originaldim, outerdim)
    
    def call(self, inputs):
        zmean, zvar, z = self.encoders(inputs)
        reconstructed = self.decoders(z)

        kl_loss = -0.5 * tf.reduce_mean(zvar, tf.square(zmean) - tf.exp(zvar) + 1)

        self.add_loss(kl_loss)
        return reconstructed

In [ ]:
(xtrn, _), (_, _) = tf.keras.datasets.mnist.load_data()
xtrn = xtrn.reshape(60000, 784).astype("float32") / 255
trn_data = tf.data.Dataset.from_tensor_slices(xtrn)
trn_data = trn_data.shuffle(1024).batch(64)

In [ ]:
originaldim = 748
outerdim = 64
innerdim = 32

mse_loss = tf.keras.losses.MeanSquaredError()
loss_metric = tf.keras.metrics.Mean()
optimizer = tf.keras.optimizers.Adam(1e-3)
vae = VariationAutoEncoder(784, 64, 32)

epochs = 2

for epoch in range(epochs):
    print(f"epoch: {epoch}")
    for steps, x_batch_train in enumerate(trn_data):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            loss = mse(x_batch_train, reconstructed)
            loss += sum(vae.losses)
        
        grad = tape.gradient(loss, vae.trainable_variables)
        optimizer.apply_gradients(zip(grad, vae.trainable_variables))

        loss_metric(loss)

        if steps % 100 == 0:
            print("step %d: mean loss = %0.4f" % (steps, loss_metric.result()))

epoch: 0


TypeError: ignored

In [ ]:
tf.__version__